In [ ]:
from imports import *
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:,.2f}'.format
plt.rc('figure', max_open_warning=200)
%matplotlib notebook
sns.set_theme()

In [ ]:
# Path for loading EEG raw data
init_dir_path = os.path.join(os.getcwd(), 'work_data')
data_raw = mne.io.read_raw_edf(init_dir_path + '/pigarev_sleep.edf', preload=True)
sampling_rate = data_raw.info['sfreq']
data_raw.drop_channels(['E', 'DC01', 'DC02', 'DC03', 'DC04', 'DC05', 'DC06', 'DC07', 'DC08', 'DC09', 'DC10', 'DC11', 'DC12', 'DC13', 'DC14', 'DC15', 'DC16', 'EEG Mark1', 'EEG Mark2', 'FTT7h', 'TPP7h', 'FFC5h', 'CCP5h', 'FCC3h', 'CPP3h', 'PPO9h', 'OI1h', 'I1', 'CCP1h', 'PPO1h', 'AFF1h', 'FFC1h', 'FFC2h', 'AFF2h', 'PPO2h', 'CCP2h', 'I2', 'OI2h', 'PPO10h', 'CPP4h', 'FCC4h', 'CCP6h', 'FFC6h', 'TPP8h', 'FTT8h', 'TTP7h', 'TPP9h', 'FTT9h', 'FFT9h', 'FFT7h', 'AFF5h', 'FCC5h', 'FFC3h', 'CCP3h', 'CPP5h', 'POO9h', 'POO5h', 'CPP1h', 'POO1', 'AFp1', 'FCC1h', 'FCC2h', 'AFp2', 'POO2', 'CPP2h', 'POO6h', 'POO10h', 'CPP6h', 'CCP4h', 'FFC4h', 'FCC6h', 'AFF6h', 'FFT8h', 'FFT10h', 'FTT10h', 'TTP8h', 'TPP10h', 'BP1', 'BP2', 'BP3', 'BP4', 'Events/Markers'])


In [ ]:
rois = [['Fp1','Fp2','Fpz'],['AF7, AF3','AF4','AF8'], ['F9','F7','F5','F3','FT9','FT7','FC5','FC3'], 
           ['F1','Fz','F2','FC1','FCz','FC2'],
           ['F4','F6','F8','F10','FC4','FC6','FT8','FT10'], 
           ['T7','TP9','TP7'],
           ['T8','TP8','TP10'], 
           ['C5','C3','CP5','CP3'], 
           ['C1','Cz','C2','CP1','CPz', 'CP2'], 
           ['C4','C6','CP4','CP6'], 
           ['P9','P7','P5','P3'],
           ['P1','Pz','P2'],
           ['P4','P6','P8','P10'],
           ['PO9','PO7','PO3','O1'], 
           ['POz','Oz'],
           ['PO4','PO8','PO10','O2']]

In [ ]:

data_raw.set_montage('standard_1020')
fig = data_raw.plot_sensors(ch_type='eeg', show_names=True, sphere="eeglab", ch_groups=rois)


# Re-referencing raw data (common average reference)
bands = [(0.9, 4, 'Delta (0.9-4 Hz)'), (4, 8, 'Theta (4-8 Hz)'), (8, 14, 'Alpha (8-14 Hz)'), 
         (14, 25, 'Beta (14-25 Hz)'), (25, 40, 'Gamma (25-40 Hz)')]

rr_data_raw, ref_ch = mne.set_eeg_reference(data_raw, ref_channels='average', copy=False, projection=False, ch_type='auto')

rr_data_raw_filt = rr_data_raw.filter(l_freq=bands[0][0], h_freq=bands[-1][1])
# rr_data_raw_filt.plot(duration=5)

# fig = rr_data_raw_filt.plot_psd(fmin=0, fmax=50)
# fig.savefig('filtered_reref_psd.png')
# print()

In [ ]:
rr_data_ica = rr_data_raw_filt
rr_data_ica.set_montage('standard_1020')
rr_ica = ICA(n_components=.99, max_iter='auto', random_state=97, method='infomax')
rr_ica.fit(rr_data_ica)

In [ ]:
fig_sources = rr_ica.plot_sources(rr_data_ica)
fig_sources.savefig('ica_sources.png')
print()
fig_comps = rr_ica.plot_components()
print()

In [ ]:
muscle_idx_auto, scores = rr_ica.find_bads_muscle(rr_data_ica)
# rr_ica.plot_scores(scores, exclude=muscle_idx_auto)
# muscle_idx_auto
eye_idx_auto, scores = rr_ica.find_bads_eog(rr_data_ica, ch_name='Fp1')
# rr_ica.plot_scores(scores, exclude=eye_idx_auto)
# eye_idx_auto
rr_ica.apply(rr_data_ica, exclude=muscle_idx_auto + eye_idx_auto)
# rr_ica.plot_overlay(rr_data_ica, exclude=muscle_idx_auto + eye_idx_auto)
# print()
# rr_data_ica.plot_psd(fmin = 0, fmax=105)
# print()

In [ ]:
#save

rr_data_ica.save('reref_filterd_raw.fif')